# Project 1: Exploratory Data Analysis
Diabetes Data

First Name, Last Name

## Declaration

**The source of the data is the *The Juvenile Diabetes Research Foundation 
Continuous Glucose Monitoring Study Group*, but the analyses, content 
and conclusions presented herein are solely the responsibility of the authors 
and have not been reviewed or approved by the aforementioned study ownwers.**

# Data Notes

- Enter your own notes here about things you want to jot down, keep in mind


## Edge cases
-


## Caveats / Assumptions
- 

# Python Imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [14]:
import pandas as pd # ensure v = 0.24 !pip install --upgrade pandas 
import numpy as np
import datetime as dt

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap,LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches

In [27]:
import os
from pathlib import Path
import pdb

In [50]:
import dask

In [6]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [7]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [8]:
sns.set_style('darkgrid')

In [9]:
sns.set_context('talk')

# 1. Data Import

In [55]:
# Listing all data table files
directory = os.fsencode('data/')

r = []
for root, dirs, files in os.walk(directory):
    for name in files:
        r.append(os.path.join(root, name))

file_list = [f.decode('UTF-8') for f in r if 'DS_Store' not in str(f)]
file_list

['data/tblALabHbA1c.csv',
 'data/tblAPtSummary.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_6.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_7.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_5.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_4.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_1.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_3.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_2.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_12.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_10.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_11.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Blind_Baseline.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_9.csv',
 'data/CGM_data/RTCGM_group/tblADataRTCGM_Unblinded_RTCGMGroup_8.csv',
 'data/CGM_data/Control_group

In [12]:
# Import patient summary & demographics
pt_summary_orig = pd.read_csv('data/tblAPtSummary.csv')
pt_summary = pt_summary_orig.copy(deep=True)
pt_summary.head()

,RecID,PtID,Gender,AgeAsOfRandDt,Race,Ethnicity,Height,Weight,DurDiabetes,InsulinModality,NumSevHypo,HGMReadAvg,EduCareGvrP,EduCareGvrPEdu,RandDt,TxGroup
0,1,307,M,46,White,Not Hispanic or Latino,167.8,93.4,31.0,Pump,0,NaN,Subject,Bachelors,1/9/2000 13:42,Control
1,2,258,M,18,White,Not Hispanic or Latino,193.5,97.1,10.4,Pump,0,NaN,Mother,Bachelors,1/15/2000 14:26,RT-CGM
2,3,309,M,16,White,Not Hispanic or Latino,174.0,73.0,6.1,Pump,0,NaN,Mother,Bachelors,1/14/2000 9:57,Control
3,4,344,F,69,White,Not Hispanic or Latino,162.5,64.7,14.8,Pump,0,NaN,Subject,Masters,1/20/2000 11:06,Control
4,5,350,F,13,White,Not Hispanic or Latino,165.1,62.3,7.1,Pump,0,NaN,Father,Associates,1/14/2000 17:28,RT-CGM


In [52]:
# Examine single RTCGM file
cgm_test = pd.read_csv('data/CGM_data/Control_group/tblADataRTCGM_Unblinded_ControlGroup_5.csv',
                       parse_dates=['DeviceDtTm'])
cgm_test.head()

,RecID,PtID,DeviceDtTm,Glucose
0,9306853,463,2001-01-21 00:00:00,246
1,9306854,463,2001-01-21 00:05:00,242
2,9306855,463,2001-01-21 00:10:00,238
3,9306856,463,2001-01-21 00:15:00,232
4,9306857,463,2001-01-21 00:20:00,234


In [53]:
cgm_summary_test = cgm_test.merge(pt_summary,on='PtID',how='left')

In [54]:
cgm_summary_test.head()

,RecID_x,PtID,DeviceDtTm,Glucose,RecID_y,Gender,AgeAsOfRandDt,Race,Ethnicity,Height,Weight,DurDiabetes,InsulinModality,NumSevHypo,HGMReadAvg,EduCareGvrP,EduCareGvrPEdu,RandDt,TxGroup
0,9306853,463,2001-01-21 00:00:00,246,290,M,33,White,Not Hispanic or Latino,171.5,72.8,30.8,Pump,0,4.0,Subject,Bachelors,7/21/2000 12:49,Control
1,9306854,463,2001-01-21 00:05:00,242,290,M,33,White,Not Hispanic or Latino,171.5,72.8,30.8,Pump,0,4.0,Subject,Bachelors,7/21/2000 12:49,Control
2,9306855,463,2001-01-21 00:10:00,238,290,M,33,White,Not Hispanic or Latino,171.5,72.8,30.8,Pump,0,4.0,Subject,Bachelors,7/21/2000 12:49,Control
3,9306856,463,2001-01-21 00:15:00,232,290,M,33,White,Not Hispanic or Latino,171.5,72.8,30.8,Pump,0,4.0,Subject,Bachelors,7/21/2000 12:49,Control
4,9306857,463,2001-01-21 00:20:00,234,290,M,33,White,Not Hispanic or Latino,171.5,72.8,30.8,Pump,0,4.0,Subject,Bachelors,7/21/2000 12:49,Control


#### Import CGM Data

In [ ]:
cgm_control = [] # Add filenames from file_list to this list if it has 'ControlGroup' in the name
cgm_treatment = [] # Add filenames from file_list to this list if it has 'RTCGMGroup' in the name

# OR - add all CGM data together

cgm_df = [] # Add all files with "RTCGM" in name
# Note, joining all together will be very big df (consider using Dask DF)

In [ ]:
# Join ptSummary data on CGM data - with pandas, a SQL-like "join" uses the ".merge" function
cgm_control_summary = cgm_df.merge(pt_summary,on='PtID',how='left')

# 2. Data Quality Check

#### Findings:

In [ ]:
# Check for nulls (white lines in plot)
fig,ax = plt.subplots(figsize=(8,4))
sns.heatmap(cgm_control_summary.isnull(),cbar=False,ax=ax)
ax.get_yaxis().set_visible(False)
plt.show()

# 3. EDA - Feature Exploration for Driving Factors & Relationship to Outcome

In [ ]:
df.head()

In [ ]:
# Example barplot code
fig,ax = plt.subplots(figsize=(6,4))
sns.barplot(x=df['Col_X'],y=df['Outcome_metric'])
plt.show()